# *Escapa, con la llave* (Modo Automático)

---
**Lee este *Notebook* con atención**

En este *Notebook*, se va a cargar código de *Notebooks* anteriores, por lo que las funciones de los *Notebooks* anteriores deberían haber sido realizadas y **probadas**, antes de empezar con éste.

Dado que este *Notebook* carga código de *Notebooks* anteriores, cada vez que se cambie o corrija uno de los *Notebooks* anteriores, **es necesario** recargar este *Notebook* al completo.

Este *Notebook* constituye la segunda parte de la práctica.

### ¿Qué se va a hacer?

Se va a usar el algoritmo A\* para resolver el juego de *Escapa, con la llave*.

### ¿Cómo se va a hacer?

- Se va a utilizar la implementación del A\* realizada en el *Notebook* "FuncionesBusqueda". Si es necesario, habrá que corregir la implementación del A\* anteror, para que funcione. 

- Se van a implementar funciones para:
    - obtener cuándo un estado es *meta*.
    - obtener cuál es el valor heurístico de un determinado estado.
    - obtener los sucesores de un nodo. Para obtener los sucesores, se va a utilizar la función **move** realizada anteriormente. 
    
A continuación se puede ver cómo, usando la clase **DynamicCodeLoader**, se puede cargar el código de los *Notebooks* anteriores. Este ejemplo sería solamente para poder hacer pruebas, dado que la interfaz gráfica lo integra ya todo.

Todo el código del *Notebook* de la primera parte de la práctica se guarda en el módulo *model* y todo el código del *Notebook* de 'FuncionesBusqueda' se guarda en el módulo *search*.

---

In [201]:
# Cargar Celda
import os
from typing import Tuple, Callable
from DynamicCodeLoader import cargaCodigoDinamico
from EscapaConBooleanoLlave import Level, State
# atento a los nombres que tengan tus notebook en tu PC
model = cargaCodigoDinamico('JEscapaManualConBooleanoLlave.ipynb',"Model")
search = cargaCodigoDinamico('FuncionesBusquedaACompletarEstudiantes.ipynb',"Search")
nivel_global = None

### Ejemplo 2: Usar el algoritmo A\* para hacer pathfinding

En el módulo *search* tenemos una implementación de A\* y funciones y variables para hacer *pathfinding* (El problema de ir de la coordenada A a la coordenada B en un espacio de estados de coordenadas sin cajas, agua ni nada).

En el ejemplo de abajo, se crea un tablero, un estado inicial y un estado final.

La función *creaNodoInicialPF* crea un nodo para el problema del PathFinding y además inicializa los valores de las variables globales search.estadoFinalPF y search.tablero, que se usan dentro de las funciones de heurística y sucesores.

Una vez creado el nodo, se invoca a la función *search.AStar* pasandole dicho nodo y las funciones de meta, heurística y sucesores.

In [202]:
tablero = [
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 1, 0, 1, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 1, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 1, 1, 1, 1, 0, 1],
    [1, 0, 1, 0, 0, 0, 0, 1, 0, 1],
    [1, 0, 1, 0, 0, 0, 0, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 1, 0, 1],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
]



estadoInicialPF = (1,1) #y,x
estadoFinalPF = (2,5)



    

nodoInicialPF = search.creaNodoInicialPF(estadoInicialPF,
                                  estadoFinalPF,
                                  tablero,
                                  search.heuristicaPF)

solucion = search.AStar(nodoInicialPF,
                        search.sucesoresPF, 
                        search.es_metaPF, 
                        search.heuristicaPF)
print(search.dibujar(solucion, tablero))

████████████████████
██[][][]██        ██
██  ██[]██[][][][]██
██  ██[]████████[]██
██  ██[][][][]██[]██
██  ██      []██[]██
██          []██[]██
██  ██      [][][]██
██  ██            ██
████████████████████



---

## Que se necesita para resolver el juego de *Escapa, con la llave*, usando A\*

Se necesita:
- Crear un nodo (proporcionado por el profesor)
- Saber cuando un estado es meta (proporcionado por el profesor)
- Función heurística (**Implementado por el alumno**)
- Función de sucesores (**Implementado por el alumno**)

---


#### Crear un nodo

La función **nodo_inicial_JEscapa** crea un nodo inicial se Kwirk con el estado y el nivel especificado. Usa la función fH para calcular el valor heurístico.

Además inicializa:
- nivel_global. Una variable global que puede ser usada por las funciones de meta, de hurística y de sucesores
- num_evaluados. Una variable que lleva la cuenta del número de nodos evaluados. Y saber posteriormente que implementación de heurística es mejor.

In [203]:
# Cargar Celda
num_evaluados = 0
coste_total = 0


def nodo_inicial_JEscapa(nivel_actual: Level,
                            estado_actual:
                            State,calcular_heuristica: Callable[[State],int]
                            ) -> "Node":

    """ Crea un nodo del juego de "Escapa con la llave"
    Devuelve un nodo
    Parámetros:
    nivel que contiene el tablero y los destinos
    estado inicial que contiene el jugador y las cajas
    fH heurística usada para calcular la F.
    """

    global nivel_global
    global num_evaluados
    nivel_global = nivel_actual
    print(calcular_heuristica(estado_actual))
    nodoInicial = search.Node(estado_actual,
                              None,
                              0,
                              calcular_heuristica(estado_actual))
    num_evaluados = 0 # reinicio el número de evaluados
    
    return nodoInicial

#### Saber cuando es meta

La función *meta_JEscapa* toma un nodo y va a devolver *True*, si dicho nodo contiene un estado *meta* (todas las cajas están sobre los destinos, reutilizando *es_meta*, de la primera parte).

Además, si es *meta*, recupera el valor de **G** (el coste), para saber el coste de la solución.


In [204]:
# Cargar Celda

def meta_JEscapa(nodo: "Node") -> bool:
    """ Consulta si un nodo contiene un estado meta
    Devuelve True si es meta
    Parámetros:
    nodo a evaluar
    """ 
    estado_actual: State = nodo.get_id()
    global nivel_global
    global coste_total
    
    return estado_actual.get_player() is None

### Cálculo del *valor heurístico*

**(Lo han de implementar los alumnos)**

Esta función tiene que devolver un valor númerico que estime los movimientos que *faltan*, para llegar a la meta.

Ejemplos:
- $f_0$: Devuelve siempre 0. Es *minorante*, así es que encontraría el camino más corto.
- $f_1$: Devuelve la suma de distancias de *Manhattan*, entre la meta y los jugadores. Es *minorante*.
- $f_2$ y $f_3$: Inventadas por el alumno, para que tengan en cuenta el agua y las cajas.

$f_0 < f_1$ 

Cuanto mayor sea el valor heurístico, menos nodos se explorarán, y encontrará el camino mínimo, siempre que sea *minorante*.


In [205]:
# Cargar Celda

def heuristica_JEscapa(estado: State):
    """ Evalua el coste estimado desde un estado hasta la meta
    Utiliza el método de Manhattan
    Devuelve un número positivo mayor que 0
    Parámetros:
    Estado a evaluar
    """ 
    
    global nivel_global
    # cada vez que se invoca la heurística, se incrementa en 1 el número de nodos evaluados
    global num_evaluados
    num_evaluados += 1
    print(nivel_global)
    if estado.get_player() is None:
        return 0


    return abs(nivel_global.get_destino()[0] - estado.get_player()[0]) + abs(nivel_global.get_destino()[1] - estado.get_player()[1])



### Crear sucesores

Esta función debería crear nodos *sucesores*, siguiendo una de estas dos estrategías:

*Opción sencilla*:
- Habría un máximo de 4 sucesores, el resultado de ejecutar los movimientos arriba, abajo, derecha o izquierda. Si alguno de los movimientos no se puede ejecutar, habría menos de 3 sucesores.

*Opción avanzada*: 
- Consideraríamos los siguientes tipos de sucesores:
    - Sucesores colocarse: Lleva el jugador hasta la meta, o a una posición adyacente a una caja.
    - Sucesores empujar: Empuja una caja.
    
Si se puede ir a la meta, se va a la meta. Si no, es que hay una caja o agua bloqueando el camino, así que habrá que ir junto a una caja, para así, empujarla. Los desplazamientos hasta la meta o hasta las coordenadas adyacentes a una caja se harán con la opción *pathfinding*.

Con esta idea, no se exploran estados intermedios en los que no existe ni la posibilidad de empujar una caja.

In [206]:
# Cargar Celda

def sucesores_JEscapa(nodo,hSoc):
    global nivel_global
    estado = nodo.get_id()
    g = nodo.get_g()
    hijos = []
    movs = [[1,0],[-1,0],[0,1],[0,-1]]
        
    for mov in movs:
        hijo = model.move(nivel_global,estado,mov)
        
        if hijo != estado:
            nuevaG = g+1
            h=hSoc(hijo)
            hijos.append(search.Node(hijo,nodo,nuevaG,nuevaG+h))
    
    return hijos

In [207]:
# Cargar Celda

# Crea tantas celdas como necesites, si son funciones necesarias
# para el juego deberán empezar por Cargar Celda


# Probando las funciones

La manera de probar las funciones es igual que en la primera parte de la práctica.

Se carga un nivel donde tengamos la partida que queremos probar.

In [208]:
# Si tienes problemas empieza cargando los niveles más fáciles el 0, el 1
# Aumenta la dificultad a medida que arregles los fallos.
from LoaderEscapaConBooleanoLlaveVPedro import Loader
l = Loader()
l.get_all_levels()
level_txt = open("."+os.sep+"levels"+os.sep+"level_PruebaJEscapaTrivial.txt",'r',encoding="utf-8").read()
level1, state1 = l.load_level(level_txt)
global nivel_global
nivel_global = level1

In [209]:
# Imprimimos el nivel para asegurarnos que todo es correcto.
print(level_txt)

#######
#!* ###
#x¡+ -#
#x @  #
#x    #
#######


In [210]:
level1.get_tablero()

[(0, 0, 0, 0, 0, 0, 0),
 (0, 0, 1, 1, 0, 0, 0),
 (0, 1, 1, 1, 1, 1, 0),
 (0, 1, 1, 1, 1, 1, 0),
 (0, 1, 1, 1, 1, 1, 0),
 (0, 0, 0, 0, 0, 0, 0)]

In [211]:
# ejemplo de como se visualiza un nivel
from UIJEscapaVPedro import gui
from ipywidgets import HTML
ui = gui()
htmlStr = ui.get_html(level1, state1)
HTML(value = htmlStr)

HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…

In [212]:
# Ejemplos de Model
print(state1)
print(model.is_goal(state1))

new_state1 = model.move(level1,state1,[1,0])
print(new_state1)

Player: [3, 3] + Piedra: {(2, 3)} + Agua + {(2, 5)} + Aspa: {(3, 1), (4, 1), (2, 1)} + Llave: {(2, 2)} + Tiene llave: 0
False
Player: [4, 3] + Piedra: {(2, 3)} + Agua + {(2, 5)} + Aspa: {(3, 1), (4, 1), (2, 1)} + Llave: {(2, 2)} + Tiene llave: 0


In [213]:
from UIJEscapaVPedro import gui
from ipywidgets import HTML
ui = gui()

htmlStr = ui.get_html(level1, new_state1)
HTML(value = htmlStr)

HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…

---
Se usa el estado y el nivel para para crear un Nodo. **Siempre hay que crear un nodo inicial**. Las funciones de *heurística* y *sucesores* funcionan sobre nodos.

La función de *crear nodo* sería una de las funciones proporcionadas por el profesor.

---

In [214]:
nodoInicial = nodo_inicial_JEscapa(level1,state1,heuristica_JEscapa)    
nodoInicial

Enemigo: (1, 1) + Destino: (1, 2) + Tablero: [(0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 1, 0, 0, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 0, 0, 0, 0, 0, 0)]
3
Enemigo: (1, 1) + Destino: (1, 2) + Tablero: [(0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 1, 0, 0, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 0, 0, 0, 0, 0, 0)]


Player: [3, 3] + Piedra: {(2, 3)} + Agua + {(2, 5)} + Aspa: {(3, 1), (4, 1), (2, 1)} + Llave: {(2, 2)} + Tiene llave: 0 f:3 g:0 p:p:\|

Después de crear el nodo inicial se pueden probar las funciones.


In [215]:
sucesores = sucesores_JEscapa(nodoInicial,heuristica_JEscapa)
sucesores

Enemigo: (1, 1) + Destino: (1, 2) + Tablero: [(0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 1, 0, 0, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 0, 0, 0, 0, 0, 0)]
Enemigo: (1, 1) + Destino: (1, 2) + Tablero: [(0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 1, 0, 0, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 0, 0, 0, 0, 0, 0)]
Enemigo: (1, 1) + Destino: (1, 2) + Tablero: [(0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 1, 0, 0, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 0, 0, 0, 0, 0, 0)]
Enemigo: (1, 1) + Destino: (1, 2) + Tablero: [(0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 1, 0, 0, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 0, 0, 0, 0, 0, 0)]


[Player: [4, 3] + Piedra: {(2, 3)} + Agua + {(2, 5)} + Aspa: {(3, 1), (4, 1), (2, 1)} + Llave: {(2, 2)} + Tiene llave: 0 f:5 g:1 p:p:Player: [3, 3] + Piedra: {(2, 3)} + Agua + {(2, 5)} + Aspa: {(3, 1), (4, 1), (2, 1)} + Llave: {(2, 2)} + Tiene llave: 0|,
 Player: [2, 3] + Piedra: {(1, 3)} + Agua + {(2, 5)} + Aspa: {(3, 1), (4, 1), (2, 1)} + Llave: {(2, 2)} + Tiene llave: 0 f:3 g:1 p:p:Player: [3, 3] + Piedra: {(2, 3)} + Agua + {(2, 5)} + Aspa: {(3, 1), (4, 1), (2, 1)} + Llave: {(2, 2)} + Tiene llave: 0|,
 Player: [3, 4] + Piedra: {(2, 3)} + Agua + {(2, 5)} + Aspa: {(3, 1), (4, 1), (2, 1)} + Llave: {(2, 2)} + Tiene llave: 0 f:5 g:1 p:p:Player: [3, 3] + Piedra: {(2, 3)} + Agua + {(2, 5)} + Aspa: {(3, 1), (4, 1), (2, 1)} + Llave: {(2, 2)} + Tiene llave: 0|,
 Player: [3, 2] + Piedra: {(2, 3)} + Agua + {(2, 5)} + Aspa: {(3, 1), (4, 1), (2, 1)} + Llave: {(2, 2)} + Tiene llave: 0 f:3 g:1 p:p:Player: [3, 3] + Piedra: {(2, 3)} + Agua + {(2, 5)} + Aspa: {(3, 1), (4, 1), (2, 1)} + Llave: {(2, 2)}

Podemos probar el A\* en su conjunto de la siguiente forma:

In [216]:
solucion = search.AStar(nodoInicial,
                        sucesores_JEscapa, 
                        meta_JEscapa, 
                        heuristica_JEscapa)
print("Solución ")
for estado in solucion:
    print(estado.get_id().get_player())
print("Nodos evaluados ",num_evaluados)
print("Coste de la solución ",coste_total)

Enemigo: (1, 1) + Destino: (1, 2) + Tablero: [(0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 1, 0, 0, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 0, 0, 0, 0, 0, 0)]
Enemigo: (1, 1) + Destino: (1, 2) + Tablero: [(0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 1, 0, 0, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 0, 0, 0, 0, 0, 0)]
Enemigo: (1, 1) + Destino: (1, 2) + Tablero: [(0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 1, 0, 0, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 0, 0, 0, 0, 0, 0)]
Enemigo: (1, 1) + Destino: (1, 2) + Tablero: [(0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 1, 0, 0, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 0, 0, 0, 0, 0, 0)]
Enemigo: (1, 1) + Destino: (1, 2) + Tablero: [(0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 1, 0, 0, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 1, 1, 1, 1, 1, 0), (0, 0, 0, 0, 0, 0, 0)]
Enemigo: (1, 1) + Destino: (1, 2) + Tablero: [(0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 1

Podriamos visualizar la solución de manera gráfica así:

In [217]:
# mejor en chrome
from ipywidgets import VBox, Label

htmls = []
for est in solucion:
    htmlStr = ui.get_html(level1, est.get_id()) # estoy cargando el mapa1
    htmls.append(HTML(value = htmlStr))
    


from ipywidgets import Layout, Button, VBox

box_layout = Layout(overflow_x='scroll',
                    width='310px',
                    height='',
                    flex_direction='row',
                    display='flex')
carousel = VBox(children=htmls, layout=box_layout)
VBox([carousel])

Una vez hayas implementado y probado las funciones, trata de ejecutar el juego.

El Mediador que es la clase que sirve de enganche entre las clases de interfaz y la funcionalidad, leerá las celdas que empiezan por "# Cargar Celda", cargará esas funciones dinámicamente y las usará para mover y comprobar si la partida finaliza.

In [218]:
# Utiliza el nivel "level_medio_VPedroJEscapa.txt", para encontrar la solución. 

from IPython.display import display
from MediadorVPedroConBooleanoLlave import Mediator
from UIJEscapaVPedro import gui

import warnings
warnings.filterwarnings("ignore")
ui = gui(manual = False)

med = Mediator.get_instance(modelPath = "JEscapaManualConBooleanoLlave.ipynb",
                           aStarPath = 'FuncionesBusquedaACompletarEstudiantes.ipynb',
                           nodesPath = "JuegoEscapaLlaveAutomVEstudiantes.ipynb")

med.register_ui(ui)


# Cuando llamo a la función se crean
display(ui.get_ui_elements())

NameError: name 'nivel_global' is not defined